In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [11]:
page = requests.get('https://myillini.illinois.edu/Programs')
soup = BeautifulSoup(page.content, 'html.parser')
majors = soup.find('div', class_='columns-2').find_all('p')

In [12]:
baseUrl = 'https://myillini.illinois.edu/Programs/MajorDetail/'
baseSampleJob = 'https://myillini.illinois.edu/Programs/ProgramSurveyEmployers/'
baseSampleGradSchool = 'https://myillini.illinois.edu/Programs/ProgramSurveyGradSchools/'

urls = []
for major in majors:
    urls.append(list(major.children)[1]['href'].split('/')[3])

In [13]:
# Do not arbitrarily change the order of this.
fieldsKey = [
    'Major Name',
    'Major Description',
    'College',
    'Related Majors',
    'Career Options',
    'Avg Starting Salary',
    'Sample Employer Destinations',
    'Sample Grad School Destinations',
    'Employed After Grad',
    'Attending Grad School',
    'Years To Degree',
    '4 Yr Grad Rate',
    'International Tuition',
    'Non-Resident Tuition',
    'Illinois Tuition',
    'Fees',
    'Expenses'
]

In [14]:
info = pd.DataFrame(columns=fieldsKey)

In [15]:
for majorCode in urls:
    url = baseUrl + majorCode
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    data = {}
    print(soup.find('div', class_='programItem').find('h1').get_text())

    try:
        data[fieldsKey[0]] = soup.find('div', class_='programItem').find('h1').get_text()
    except: 
        pass 
    try:
        data[fieldsKey[1]] = soup.find('div', class_='programDescription').find('p').get_text()
    except: 
        pass 
    try:
        data[fieldsKey[2]] = soup.find('div', class_='programUrls').find('a').get_text()
    except: 
        pass 
    try:
        data[fieldsKey[3]] = [x.find('div').find('a').get_text() for x in list(soup.find('div', class_="relatedMajors").find_all('div', class_='data-parent major-holder'))]
    except: 
        pass 
    
    money = soup.find_all('ul', class_="list-group list-group-flush tuition-list mb-4")
    try:
        data[fieldsKey[12]] = soup.find('ul', class_="list-group list-group-flush tuition-list").find('li').find_all('span')[1].get_text()
    except: 
        pass
    try:
        data[fieldsKey[13]] = money[1].find('li').find_all('span')[1].get_text()
    except: 
        pass 
    try:
        data[fieldsKey[14]] = money[0].find('li').find_all('span')[1].get_text()
    except: 
        pass 
    try:
        data[fieldsKey[15]] = money[1].find_all('li')[1].find_all('span')[1].get_text()
    except: 
        pass 
    try:
        data[fieldsKey[16]] = money[1].find_all('li')[2].find_all('span')[1].get_text()
    except: 
        pass 
    
    numbers = soup.find_all('span', class_="number")
    try:
        data[fieldsKey[8]] = numbers[1].get_text()
    except: 
        pass 
    try:
        data[fieldsKey[9]] = numbers[2].get_text()
    except: 
        pass 
    if len(numbers) == 8:
        try:
            data[fieldsKey[5]] = numbers[3].get_text() 
        except: 
            pass 
        try:
            data[fieldsKey[10]] = numbers[4].get_text() 
        except: 
            pass 
        try:
            data[fieldsKey[11]] = numbers[6].get_text()
        except: 
            pass 
    elif len(numbers) == 9:
        try:
            data[fieldsKey[5]] = numbers[4].get_text() 
        except: 
            pass 
        try:
            data[fieldsKey[10]] = numbers[5].get_text() 
        except: 
            pass 
        try:
            data[fieldsKey[11]] = numbers[7].get_text()
        except: 
            pass 
    elif len(numbers) == 6:
        try:
            data[fieldsKey[5]] = numbers[3].get_text() 
        except: 
            pass 
        try:
            data[fieldsKey[10]] = numbers[4].get_text() 
        except: 
            pass 
        try:
            data[fieldsKey[11]] = numbers[7].get_text()
        except: 
            pass 
    elif len(numbers) == 4:
        try:
            data[fieldsKey[5]] = NaN
        except: 
            pass 
        try:
            data[fieldsKey[10]] = numbers[0].get_text() 
        except: 
            pass 
        try:
            data[fieldsKey[11]] = numbers[2].get_text()
        except: 
            pass

    try:
        sampleJobs = baseSampleJob + majorCode
        sampleGradSchool = baseSampleGradSchool + majorCode
        res2 = requests.get(sampleJobs)
        res3 = requests.get(sampleGradSchool)
        soup2 = BeautifulSoup(res2.content, 'html.parser')
        soup3 = BeautifulSoup(res3.content, 'html.parser')
        all2 = soup2.find_all('li')
        all3 = soup3.find_all('li')        
        data[fieldsKey[6]] = [x.get_text() for x in all2]
        data[fieldsKey[7]] = [x.get_text() for x in all3]
    except:
        pass

    try:
        data[fieldsKey[4]] = [x.get_text() for x in list(soup.find('div', class_="programCareerExamples").find_all('li'))]
    except: 
        pass 
    info = info.append(data, ignore_index=True)

Accountancy
ACES Undeclared
Acting
Actuarial Science
Advertising
Aerospace Engineering
African American Studies
Agri-Accounting
Agribusiness, Markets & Management
Agricultural and Biological Engineering
Ag & Bio Engineering
Agricultural and Consumer Economics
Agricultural Communications - Advertising
Agricultural Communications - Journalism
Agricultural Education
Agricultural Leadership, Education, and Communications
Animal Sciences
Anthropology
Architectural Studies
Art and Art History 
Art Education (K-12)
Arts and Entertainment Technology 
Asian American Studies
Astronomy
Atmospheric Sciences
Biochemistry
Bioengineering
Biology
Brain and Cognitive Science
Chemical Engineering
Chemistry
Civil Engineering
Classics
Communication
Community Health
Comparative and World Literature
Computer Engineering
Computer Science
Computer Science and Advertising
Computer Science and Animal Sciences
Computer Science and Anthropology
Computer Science and Astronomy
Computer Science and Chemistry
Compute

In [16]:
info['4 Yr Grad Rate']

0      83%
1      74%
2      60%
3      85%
4      80%
      ... 
164    NaN
165    74%
166    60%
167    NaN
168    63%
Name: 4 Yr Grad Rate, Length: 169, dtype: object

In [17]:
# TODO

# Show pie chart of which colleges the different companies hire from (section size to correspond to number of majors hired for)
# Draw bar graph showing majors for highest-earning salaries (descending order)
# Draw bar graph showing majors by employment rate (descending order)
# Create new feature calculated by salary / (100 - employed% - gradSchool%). Bar chart showing best majors with this feature.
# Calculate average salaries earned by graduates of a specific college
# Draw bi-partite graph of different companies and which majors they hire
# Draw bi-partite graph of different positions and which majors they can arise from
# Draw bi-partite graph of different companies and which college they hire from (edge weight must correspond to the relative count)
# Draw graph showing which majors are related to each other
# Calculate approximate degree cost by multiplying annual total cost with years to degree. Draw bar graph showing by major (descending order)